In [1]:
import pandas as pd, numpy as np
from tqdm import tqdm

import data_processing
import lbwsg_boe
import iron_bw_calc_industry
# import lbwsg
# import lsff_interventions
# import demography

!date
!whoami
!pwd

Wed Apr 14 23:33:24 PDT 2021
ndbs
/ihme/homes/ndbs/vivarium_research_lsff/nanosim_models


In [2]:
%load_ext autoreload
%autoreload 2

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Check what the new "industry wheat" coverage data looks like

In [3]:
!ls results_industry/

lsff_input_coverage_data_w_industry_oil_wheat_v3_04_14_2021.csv


/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
industry_coverage = pd.read_csv(
    'results_industry/lsff_input_coverage_data_w_industry_oil_wheat_v3_04_14_2021.csv', index_col=0)
industry_coverage.head()

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,location_id,location_name,sub_population,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,u5_applicable,wra_applicable
0,179,Ethiopia,NaN,maize flour,percent of population eating vehicle,na,59.263848,24.950317,93.577380,True,True
1,179,Ethiopia,NaN,maize flour,percent of population eating industrially prod...,na,27.167514,11.437632,42.897396,True,True
2,179,Ethiopia,NaN,maize flour,percent of population eating fortified vehicle,folic acid,0.000000,0.000000,0.000000,True,True
3,179,Ethiopia,NaN,maize flour,percent of population eating fortified vehicle,iron,0.000000,0.000000,0.000000,True,True
4,179,Ethiopia,NaN,maize flour,percent of population eating fortified vehicle,vitamin a,0.000000,0.000000,0.000000,True,True


In [6]:
industry_coverage.vehicle.unique()

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['maize flour', 'oil', 'wheat flour', 'industry wheat',
       'industry oil'], dtype=object)

In [7]:
industry_coverage.query("vehicle == 'industry wheat'")

,location_id,location_name,sub_population,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,u5_applicable,wra_applicable
0,168,Angola,NaN,industry wheat,percent of population eating industrially prod...,na,27.200000,17.834510,36.565490,True,True
1,161,Bangladesh,NaN,industry wheat,percent of population eating industrially prod...,na,33.101000,21.703681,44.498319,True,True
2,201,Burkina Faso,NaN,industry wheat,percent of population eating industrially prod...,na,48.000000,31.472665,64.527335,True,True
3,205,Côte d'Ivoire,NaN,industry wheat,percent of population eating industrially prod...,na,90.160000,59.116156,100.000000,True,True
4,202,Cameroon,NaN,industry wheat,percent of population eating industrially prod...,na,65.440000,42.907733,87.972267,True,True
...,...,...,...,...,...,...,...,...,...,...,...
120,213,Niger,NaN,industry wheat,percent of population eating fortified vehicle,zinc,30.000000,14.760180,45.239820,True,True
121,522,Sudan,NaN,industry wheat,percent of population eating fortified vehicle,folic acid,6.525113,3.210395,9.839831,True,True
122,522,Sudan,NaN,industry wheat,percent of population eating fortified vehicle,iron,6.525113,3.210395,9.839831,True,True
123,522,Sudan,NaN,industry wheat,percent of population eating fortified vehicle,vitamin a,6.525113,3.210395,9.839831,True,True


In [14]:
industry_coverage.query("vehicle == 'industry wheat'").value_description.unique()

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['percent of population eating industrially produced vehicle',
       'percent of population eating fortified vehicle'], dtype=object)

In [8]:
input_data = data_processing.get_fortification_input_data()
input_data.consumption.head()

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,location_id,sub_population,vehicle,metric,mass_unit,time_unit,pop_denom,value_mean_gday,lower,upper,location_name,value_mean_coverage,value_025_percentile,value_975_percentile
0,163,total population,wheat flour,mean,g,day,capita,169.62,113.134860,226.105140,India,83.200000,79.500000,86.500000
1,179,total population,maize flour,mean,g,day,capita,113.84,68.762191,158.917809,Ethiopia,59.263848,24.950317,93.577380
2,179,total population,wheat flour,mean,g,day,capita,97.64,65.124913,130.155087,Ethiopia,28.000000,13.776168,42.223832
3,214,total population,maize flour,mean,g,day,capita,95.10,57.442765,132.757235,Nigeria,57.415462,54.867114,60.033478
4,214,total population,wheat flour,mean,g,day,capita,49.15,32.782563,65.517437,Nigeria,62.759594,60.359594,65.129264


# Write a function to add a new "industry wheat" vehicle in the `consumption` and `concentration` input dataframes, to match the `coverage` data

All I need to do is copy the "wheat flour" data, since the consumption patterns and iron concentration won't change for industry wheat.

Note that the "Percent of population eating vehicle" also didn't change for industry wheat, so when this data gets merged into the consumption dataframe from the wheat flour coverage data, I don't need to worry about changing that either, just copy it.

In [9]:
def append_duplicate_vehicle_data(df, vehicle, duplicated_vehicle_name):
    vehicle_df = df.query("vehicle == @vehicle").assign(vehicle=duplicated_vehicle_name)
    return pd.concat([df, vehicle_df], ignore_index=True)

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
print(input_data.consumption.shape)
print(input_data.consumption.query("vehicle=='wheat flour'").shape)

(42, 14)
(25, 14)


In [12]:
df = append_duplicate_vehicle_data(
    input_data.consumption, 'wheat flour', 'industry wheat')
df.shape

(67, 14)

In [13]:
df.query("vehicle == 'industry wheat'")

,location_id,sub_population,vehicle,metric,mass_unit,time_unit,pop_denom,value_mean_gday,lower,upper,location_name,value_mean_coverage,value_025_percentile,value_975_percentile
42,163,total population,industry wheat,mean,g,day,capita,169.62000,113.134860,226.105140,India,83.200000,79.500000,86.500000
43,179,total population,industry wheat,mean,g,day,capita,97.64000,65.124913,130.155087,Ethiopia,28.000000,13.776168,42.223832
44,214,total population,industry wheat,mean,g,day,capita,49.15000,32.782563,65.517437,Nigeria,62.759594,60.359594,65.129264
45,161,women of reproductive age,industry wheat,mean,g,day,capita,77.90000,51.958528,103.841472,Bangladesh,41.900000,31.257526,52.542474
46,165,women of reproductive age,industry wheat,mean,g,day,capita,86.17526,57.478045,114.872475,Pakistan,99.999980,99.999970,99.999990
47,189,total population,industry wheat,mean,g,day,capita,32.96000,21.983994,43.936006,United Republic of Tanzania,51.500000,44.500000,58.500000
48,190,total population,industry wheat,mean,g,day,capita,30.27000,20.189790,40.350210,Uganda,11.200000,7.700000,14.700000
49,196,total population,industry wheat,mean,g,day,capita,156.36000,104.290571,208.429429,South Africa,10.917484,6.391078,15.990590
50,201,total population,industry wheat,mean,g,day,capita,37.64000,24.468022,50.811978,Burkina Faso,48.000000,21.097889,74.902111
51,180,total population,industry wheat,mean,g,day,capita,100.11000,65.076877,135.143123,Kenya,36.580000,16.078349,57.081651


In [15]:
input_data.concentration

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Country,Food Vehicle,Indicator Value,NaFeEDTA,location_id,location_name,vehicle,value
0,Bangladesh,Wheat flour,55.000,False,161,Bangladesh,wheat flour,32.146239
1,Burkina Faso,Wheat flour,60.000,False,201,Burkina Faso,wheat flour,35.068625
2,Cameroon,Wheat flour,60.000,False,202,Cameroon,wheat flour,35.068625
3,China,Wheat flour,20.000,True,6,China,wheat flour,20.000000
4,Cote d'Ivoire,Wheat flour,30.000,False,205,Côte d'Ivoire,wheat flour,17.534312
5,Ethiopia,Wheat flour,30.000,True,179,Ethiopia,wheat flour,30.000000
6,Ghana,Wheat flour,58.500,False,207,Ghana,wheat flour,34.191909
7,India,Wheat flour,17.625,True,163,India,wheat flour,17.625000
8,Indonesia,Wheat flour,50.000,False,11,Indonesia,wheat flour,29.223854
9,Kenya,Wheat flour,40.000,True,180,Kenya,wheat flour,40.000000


In [16]:
df2 = append_duplicate_vehicle_data(
    input_data.concentration, 'wheat flour', 'industry wheat')
df2.shape

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(42, 8)

In [17]:
df2

,Country,Food Vehicle,Indicator Value,NaFeEDTA,location_id,location_name,vehicle,value
0,Bangladesh,Wheat flour,55.000,False,161,Bangladesh,wheat flour,32.146239
1,Burkina Faso,Wheat flour,60.000,False,201,Burkina Faso,wheat flour,35.068625
2,Cameroon,Wheat flour,60.000,False,202,Cameroon,wheat flour,35.068625
3,China,Wheat flour,20.000,True,6,China,wheat flour,20.000000
4,Cote d'Ivoire,Wheat flour,30.000,False,205,Côte d'Ivoire,wheat flour,17.534312
5,Ethiopia,Wheat flour,30.000,True,179,Ethiopia,wheat flour,30.000000
6,Ghana,Wheat flour,58.500,False,207,Ghana,wheat flour,34.191909
7,India,Wheat flour,17.625,True,163,India,wheat flour,17.625000
8,Indonesia,Wheat flour,50.000,False,11,Indonesia,wheat flour,29.223854
9,Kenya,Wheat flour,40.000,True,180,Kenya,wheat flour,40.000000


In [18]:
consumption = pd.read_csv(f'../data_prep/outputs/lsff_gday_data.csv')
consumption

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,location_id,location_name,sub_population,vehicle,metric,mass_unit,time_unit,pop_denom,value_mean,lower,upper
0,163,India,total population,wheat flour,mean,g,day,capita,169.620000,113.134860,226.105140
1,179,Ethiopia,total population,maize flour,mean,g,day,capita,113.840000,68.762191,158.917809
2,179,Ethiopia,total population,wheat flour,mean,g,day,capita,97.640000,65.124913,130.155087
3,214,Nigeria,total population,maize flour,mean,g,day,capita,95.100000,57.442765,132.757235
4,214,Nigeria,total population,wheat flour,mean,g,day,capita,49.150000,32.782563,65.517437
5,161,Bangladesh,women of reproductive age,wheat flour,mean,g,day,capita,77.900000,51.958528,103.841472
6,165,Pakistan,women of reproductive age,wheat flour,mean,g,day,capita,86.175260,57.478045,114.872475
7,189,United Republic of Tanzania,total population,maize flour,mean,g,day,capita,172.470000,102.944854,241.995146
8,189,United Republic of Tanzania,total population,wheat flour,mean,g,day,capita,32.960000,21.983994,43.936006
9,190,Uganda,total population,maize flour,mean,g,day,capita,126.300000,75.386647,177.213353


# Now test reading all data with industry wheat added

...after incorporating the above function into my `data_processing` module.

In [19]:
input_data2 = data_processing.get_fortification_input_data(
    coverage_data_path='results_industry/lsff_input_coverage_data_w_industry_oil_wheat_v3_04_14_2021.csv',
)
input_data2.coverage.head()

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,location_id,location_name,sub_population,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,u5_applicable,wra_applicable
0,0,179,Ethiopia,NaN,maize flour,percent of population eating vehicle,na,59.263848,24.950317,93.577380,True,True
1,1,179,Ethiopia,NaN,maize flour,percent of population eating industrially prod...,na,27.167514,11.437632,42.897396,True,True
2,2,179,Ethiopia,NaN,maize flour,percent of population eating fortified vehicle,folic acid,0.000020,0.000010,0.000030,True,True
3,3,179,Ethiopia,NaN,maize flour,percent of population eating fortified vehicle,iron,0.000020,0.000010,0.000030,True,True
4,4,179,Ethiopia,NaN,maize flour,percent of population eating fortified vehicle,vitamin a,0.000020,0.000010,0.000030,True,True


In [20]:
input_data2.concentration

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Country,Food Vehicle,Indicator Value,NaFeEDTA,location_id,location_name,vehicle,value
0,Bangladesh,Wheat flour,55.000,False,161,Bangladesh,wheat flour,32.146239
1,Burkina Faso,Wheat flour,60.000,False,201,Burkina Faso,wheat flour,35.068625
2,Cameroon,Wheat flour,60.000,False,202,Cameroon,wheat flour,35.068625
3,China,Wheat flour,20.000,True,6,China,wheat flour,20.000000
4,Cote d'Ivoire,Wheat flour,30.000,False,205,Côte d'Ivoire,wheat flour,17.534312
5,Ethiopia,Wheat flour,30.000,True,179,Ethiopia,wheat flour,30.000000
6,Ghana,Wheat flour,58.500,False,207,Ghana,wheat flour,34.191909
7,India,Wheat flour,17.625,True,163,India,wheat flour,17.625000
8,Indonesia,Wheat flour,50.000,False,11,Indonesia,wheat flour,29.223854
9,Kenya,Wheat flour,40.000,True,180,Kenya,wheat flour,40.000000


In [21]:
input_data2.consumption

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,location_id,sub_population,vehicle,metric,mass_unit,time_unit,pop_denom,value_mean_gday,lower,upper,location_name,value_mean_coverage,value_025_percentile,value_975_percentile
0,163,total population,wheat flour,mean,g,day,capita,169.62,113.134860,226.105140,India,83.200000,79.500000,86.500000
1,179,total population,maize flour,mean,g,day,capita,113.84,68.762191,158.917809,Ethiopia,59.263848,24.950317,93.577380
2,179,total population,wheat flour,mean,g,day,capita,97.64,65.124913,130.155087,Ethiopia,28.000000,13.776168,42.223832
3,214,total population,maize flour,mean,g,day,capita,95.10,57.442765,132.757235,Nigeria,57.415462,54.867114,60.033478
4,214,total population,wheat flour,mean,g,day,capita,49.15,32.782563,65.517437,Nigeria,62.759594,60.359594,65.129264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,141,total population,industry wheat,mean,g,day,capita,399.23,259.520941,538.939059,Egypt,88.484772,43.535039,99.999990
63,207,total population,industry wheat,mean,g,day,capita,86.36,56.138638,116.581362,Ghana,82.658625,40.668539,99.999990
64,181,total population,industry wheat,mean,g,day,capita,35.37,22.992400,47.747600,Madagascar,45.179608,22.228638,68.130578
65,213,total population,industry wheat,mean,g,day,capita,11.45,7.443115,15.456885,Niger,41.424994,20.381346,62.468643


In [22]:
input_data2.consumption.metric.unique()

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['mean', 'median'], dtype=object)

In [23]:
input_data2.consumption.query("metric=='median'")

,location_id,sub_population,vehicle,metric,mass_unit,time_unit,pop_denom,value_mean_gday,lower,upper,location_name,value_mean_coverage,value_025_percentile,value_975_percentile
21,202,women of reproductive age,wheat flour,median,g,day,consumers,79.0,51.354243,106.645757,Cameroon,92.0,90.0,94.0
55,202,women of reproductive age,industry wheat,median,g,day,consumers,79.0,51.354243,106.645757,Cameroon,92.0,90.0,94.0


In [24]:
input_data2.consumption.query("location_id==202")

/ihme/homes/ndbs/miniconda3/envs/lsff-iron-38/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,location_id,sub_population,vehicle,metric,mass_unit,time_unit,pop_denom,value_mean_gday,lower,upper,location_name,value_mean_coverage,value_025_percentile,value_975_percentile
20,202,total population,maize flour,mean,g,day,capita,139.34,82.246590,196.433410,Cameroon,47.295467,19.911581,74.679353
21,202,women of reproductive age,wheat flour,median,g,day,consumers,79.00,51.354243,106.645757,Cameroon,92.000000,90.000000,94.000000
55,202,women of reproductive age,industry wheat,median,g,day,consumers,79.00,51.354243,106.645757,Cameroon,92.000000,90.000000,94.000000
